# Data Retrieval

In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor

In [2]:
# File path for cities: San Francisco(sf), 
# Okaland(ok), Santa Clara(sc), Santa Cruze(scz)

file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}/calendar.csv'

sf_file = file_url.format('sf_airbnb')
ok_file = file_url.format('ok_airbnb')
sc_file = file_url.format('sc_airbnb')
scz_file = file_url.format('scz_airbnb')

In [3]:
# Loaded price by date information for each city

pricing_sf = pd.read_csv(sf_file, parse_dates=['date'])
pricing_ok = pd.read_csv(ok_file, parse_dates=['date'])
pricing_sc = pd.read_csv(sc_file, parse_dates=['date'])
pricing_scz = pd.read_csv(scz_file, parse_dates=['date'])

print(pricing_sf.shape, pricing_ok.shape, pricing_sc.shape, pricing_scz.shape)

(2420680, 4) (1057770, 4) (2068820, 4) (573050, 4)


In [4]:
# Concatenated four files into one
# Created keys for each file for future references

frames = [pricing_sf, pricing_ok, pricing_sc, pricing_scz]
keys = ['pricing_sf', 'pricing_ok', 'pricing_sc', 'pricing_scz']

df_pricing = pd.concat(frames, keys=keys)

df_pricing.head(10)

listing_id       date available price
pricing_sf 0         958 2019-08-05         f   NaN
           1         958 2019-08-04         f   NaN
           2         958 2019-08-03         f   NaN
           3         958 2019-08-02         f   NaN
           4         958 2019-08-01         f   NaN
           5         958 2019-07-31         f   NaN
           6         958 2019-07-30         f   NaN
           7         958 2019-07-29         f   NaN
           8         958 2019-07-28         f   NaN
           9         958 2019-07-27         f   NaN

# Data Exploration

## Smaller Set of Data

In [5]:
# Filter to houses that are available and have listing prices

df_pricing_smaller = df_pricing[df_pricing['available']=='t']
df_pricing_smaller = df_pricing_smaller.copy()

print('Dimension for df_pricing_smaller: ', df_pricing_smaller.shape)
print ('\nNumber of Unique ids: ', df_pricing_smaller.listing_id.nunique())

Dimension for df_pricing_smaller:  (2853357, 4)

Number of Unique ids:  14770


In [6]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [7]:
display( df_pricing_smaller.date.describe(include='all'))

count                 2853357
unique                    389
top       2018-11-03 00:00:00
freq                    10393
first     2018-08-06 00:00:00
last      2019-08-29 00:00:00
Name: date, dtype: object

## Data Format

In [10]:
def apply2df(df, column_names, function):
    """
    This enables various functions to be applied to multiple columns of dataframe.
    Input: dataframe, columns to apply function to, function
    Return: updated dataframe
    """
    for column in column_names:
        df[column] = df[column].apply(lambda x: function(x))
    return df

### Currency

In [9]:
# Noted currencies are formated as strings and need to be converted to float

def currency2float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [11]:
# Converted price to float format

curriencies = ['price']

df_pricing_smaller = apply2df(df_pricing_smaller, curriencies, currency2float)

### Outliers

In [13]:
percentiles = [0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]

df_pricing_smaller.price.describe(percentiles=percentiles)

count    2.853357e+06
mean     2.132333e+02
std      7.035042e+02
min      1.000000e+01
5%       4.100000e+01
10%      5.400000e+01
25%      8.000000e+01
50%      1.350000e+02
75%      2.250000e+02
90%      3.850000e+02
95%      5.500000e+02
max      1.024520e+05
Name: price, dtype: float64

In [14]:
# Removed outliers based on 'price'. Use 5% and 95% percentiles as cutoff

five_percentile = np.percentile(df_pricing_smaller.price, 5)
ninety_five_percentile = np.percentile(df_pricing_smaller.price, 95)

df_pricing_smaller = df_pricing_smaller[(
                        df_pricing_smaller.price>=five_percentile)
                        & (df_pricing_smaller.price<=ninety_five_percentile)]

In [15]:
print ('df_pricing_smaller dimensions: ', df_pricing_smaller.shape)

df_pricing_smaller dimensions:  (2573002, 4)


## Seasonality Data

In [16]:
# Added 'month' column to extract month of date
# Added 'day_of_week' column to extact day of week. 
# Default format: Monday=0, Sunday=6

df_pricing_smaller['month'] = df_pricing_smaller['date'].dt.month
df_pricing_smaller['day_of_week'] = df_pricing_smaller['date'].dt.weekday


print('df_pricing_smaller dimensions', df_pricing_smaller.shape)
display(df_pricing_smaller.head())

df_pricing_smaller dimensions (2573002, 6)


listing_id       date available  price  month  day_of_week
pricing_sf 63         958 2019-05-02         t  181.0      5            3
           64         958 2019-05-01         t  181.0      5            2
           65         958 2019-04-30         t  181.0      4            1
           66         958 2019-04-29         t  181.0      4            0
           67         958 2019-04-28         t  181.0      4            6

In [17]:
df_pricing_smaller.isnull().sum()

listing_id     0
date           0
available      0
price          0
month          0
day_of_week    0
dtype: int64

In [26]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price', 'month', 'day_of_week'], dtype='object')

In [27]:
columns2display = ['date', 'price', 'month', 'day_of_week']

df_pricing_smaller[columns2display].describe()

,price,month,day_of_week
count,2.573002e+06,2.573002e+06,2.573002e+06
mean,1.641822e+02,6.556472e+00,2.987381e+00
std,1.056804e+02,3.633767e+00,2.003119e+00
min,4.100000e+01,1.000000e+00,0.000000e+00
25%,8.500000e+01,3.000000e+00,1.000000e+00
50%,1.350000e+02,7.000000e+00,3.000000e+00
75%,2.100000e+02,1.000000e+01,5.000000e+00
max,5.500000e+02,1.200000e+01,6.000000e+00


### Option 1

In [28]:
df_by_month_day = df_pricing_smaller.groupby(['listing_id', 'month', 'day_of_week'])  \
                                    .price.mean()  \
                                    .reset_index()
df_by_month_day.head()

,listing_id,month,day_of_week,price
0,958,1,0,180.0
1,958,1,1,179.5
2,958,1,2,179.5
3,958,1,3,179.0
4,958,1,4,183.0


In [29]:
df_month_day_counts = df_by_month_day.groupby(['listing_id'])  \
                                     .price.count()  \
                                 .reset_index()
df_month_day_counts.rename(columns = {'price':'counts'}, inplace=True)
df_month_day_counts.head()

,listing_id,counts
0,958,48
1,3083,35
2,3264,21
3,5739,12
4,5858,84


In [30]:
# Find out listings that have full price information (12 months and 7 days/week)

listing_with_12m_7d_price = list(df_month_day_counts[
                                df_month_day_counts.counts==12*7]
                                .listing_id)

In [31]:
df_pricing_available = df_pricing_smaller[df_pricing_smaller.listing_id.isin(
                                          listing_with_12m_7d_price)]
df_pricing_available.shape

(992624, 6)

In [32]:
df_pricing_available.describe(include='all')

,listing_id,date,available,price,month,day_of_week
count,9.926240e+05,992624,992624,992624.000000,992624.000000,992624.000000
unique,NaN,389,1,NaN,NaN,NaN
top,NaN,2019-05-06 00:00:00,t,NaN,NaN,NaN
freq,NaN,2849,992624,NaN,NaN,NaN
first,NaN,2018-08-06 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2019-08-29 00:00:00,NaN,NaN,NaN,NaN
mean,1.415364e+07,NaN,NaN,165.434921,6.429842,2.992619
std,8.658768e+06,NaN,NaN,108.560755,3.485755,2.003589
min,5.858000e+03,NaN,NaN,41.000000,1.000000,0.000000
25%,6.649234e+06,NaN,NaN,80.000000,3.000000,1.000000


In [39]:
# Group by month by weekday, and calculate an average pricing

df_pricing_by_month_weekday = df_pricing_available.groupby(  \
                              ['listing_id', 'month', 'day_of_week']) \
                              .price.mean()  \
                              .reset_index()
df_pricing_by_month_weekday.head(100)

,listing_id,month,day_of_week,price
0,5858,1,0,235.0
1,5858,1,1,235.0
2,5858,1,2,235.0
3,5858,1,3,235.0
4,5858,1,4,235.0
5,5858,1,5,235.0
6,5858,1,6,235.0
7,5858,2,0,235.0
8,5858,2,1,235.0
9,5858,2,2,235.0


In [34]:
df_pricing_by_month_weekday.describe(include='all')

,listing_id,month,day_of_week,price
count,2.393160e+05,239316.00000,239316.000000,239316.000000
mean,1.413167e+07,6.50000,3.000000,165.349581
std,8.659318e+06,3.45206,2.000004,108.294469
min,5.858000e+03,1.00000,0.000000,41.000000
25%,6.591359e+06,3.75000,1.000000,80.000000
50%,1.435178e+07,6.50000,3.000000,132.500000
75%,2.203813e+07,9.25000,5.000000,220.000000
max,2.815247e+07,12.00000,6.000000,550.000000


In [35]:
# Check for null values if any

df_pricing_by_month_weekday.isnull().sum()

listing_id     0
month          0
day_of_week    0
price          0
dtype: int64

In [36]:
# Rename the average price column to avoid confusion

df_pricing_by_month_weekday.rename(columns={'price':'price_by_month_week'}, inplace=True)

In [37]:
df_pricing_by_month_weekday.shape

(239316, 4)

In [38]:
df_pricing_by_month_weekday.head()

,listing_id,month,day_of_week,price_by_month_week
0,5858,1,0,235.0
1,5858,1,1,235.0
2,5858,1,2,235.0
3,5858,1,3,235.0
4,5858,1,4,235.0


### option 2

In [ ]:
df_pricing_by_month_weekday = df_pricing_smaller

In [40]:
def get_month(number):
    """
    Given a number, return the name of month.
    
    """
    month_lookup = {1:'Jan.', 2:'Feb.', 3:'Mar.', 4:'Apr.', 5:'May.', 6:'Jun.',
                    7:'Jul.', 8:'Aug.', 9:'Sep.', 10:'Oct.', 11:'Nov.', 12:'Dec.'}
    
    return month_lookup[number]

In [41]:
def get_weekdays(number):
    """
    Given a number, return the name of month.
    
    """
    weekday_lookup = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 
                      4:'Friday', 5:'Saturday', 6:'Sunday'}
    
    return weekday_lookup[number]

In [43]:
df_pricing_by_month_weekday['month'] = df_pricing_by_month_weekday['month']  \
                                       .apply(lambda x: get_month(x))

In [44]:
df_pricing_by_month_weekday['day_of_week'] = df_pricing_by_month_weekday['day_of_week']  \
                                             .apply(lambda x: get_weekdays(x))

In [45]:
df_pricing_final = pd.get_dummies(df_pricing_by_month_weekday, 
                                  prefix='',
                                  prefix_sep='',
                                  columns=['month', 'day_of_week'])

In [46]:
df_pricing_final.columns

Index(['listing_id', 'price', 'Apr.', 'Aug.', 'Dec.', 'Feb.', 'Jan.', 'Jul.',
       'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'],
      dtype='object')

In [47]:
df_pricing_final.shape

(239316, 21)

In [48]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'

listings = pkl_file.format('df_listings_selected_features')

df_listings_reviews = pd.read_pickle(listings)
df_listings_reviews.head()

,id,bedrooms,cleaning_fee,room_type_Entire home/apt,city_Oakland,reviews_per_month,accommodates,bathrooms,city_San Francisco,number_of_reviews,...,amenities_Kitchen,amenities_Essentials,amenities_Stepfree access,amenities_Pets live on this property,amenities_Host greets you,neighbourhood_cleansed_Unincorporated Areas,neighbourhood_cleansed_Noe Valley,amenities_Coffee maker,ref_dist,ref_juri
0,958,1.0,100.0,1,0,1.38,3,1.0,1,152,...,1,1,0,1,0,0,0,0,San Francisco,SAN FRANCISCO
1,5858,2.0,100.0,1,0,0.99,5,1.0,1,112,...,1,1,0,0,0,0,0,0,San Francisco,SAN FRANCISCO
2,7918,1.0,50.0,0,0,0.16,2,4.0,1,17,...,1,0,0,0,1,0,0,0,San Francisco,SAN FRANCISCO
3,8142,1.0,50.0,0,0,0.15,2,4.0,1,7,...,1,0,0,0,1,0,0,0,San Francisco,SAN FRANCISCO
4,8567,2.0,125.0,1,0,0.27,6,1.0,1,30,...,1,1,0,0,0,0,0,1,San Francisco,SAN FRANCISCO


In [49]:
df_listings_reviews.shape

(10106, 103)

In [ ]:
df_final = pd.merge(left=df_pricing_final, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [ ]:
df_final.shape

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final.shape

In [ ]:
df_final.columns

In [ ]:
# Drop duplicate columns

df_final.drop(['id'], axis=1, inplace=True)

In [ ]:
df_final.shape

In [ ]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

to_drop

In [ ]:
# Drop features that have high correlation

df_final.drop(to_drop, axis=1, inplace=True)

In [ ]:
df_final.shape

In [ ]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')

with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)

In [ ]:
file_updated = True
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')


if file_updated:
    with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)
else:
    try: 
        with open(pkl_path, 'rb') as file:
            df_final = pd.read_pickle(file)
    except:
        with open(pkl_path, 'wb') as file:
            pd.to_pickle(df_final, file)

In [ ]:
# Perform multiple modeling for price prediction

X = df_final.drop(['price', 'date', 'available','ref_district', 'ref_jurisdiction'], axis=1)
y = df_final.loc[:, ['listing_id', 'price']]

In [ ]:
grp_by_listing_ids = df_final.groupby('listing_id').first().reset_index().listing_id

In [ ]:
listing_ids = list (grp_by_listing_ids)

In [ ]:
num_of_grps = len(grp_by_listing_ids)

training_ids = np.random.choice(listing_ids, int(num_of_grps*0.75), replace=False)
print(len(training_ids))
training_ids

In [ ]:
testing_ids = set(grp_by_listing_ids) - set (training_ids)
len(testing_ids)

In [ ]:
X_train = X[X['listing_id'].isin(training_ids)].drop('listing_id', axis=1)
X_test = X[X['listing_id'].isin(testing_ids)].drop('listing_id', axis=1)
y_train = y[y['listing_id'].isin(training_ids)].drop('listing_id', axis=1).values.ravel()
y_test = y[y['listing_id'].isin(testing_ids)].drop('listing_id', axis=1).values.ravel()

print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [ ]:
# Model-LR

model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

In [ ]:
# Model-Random Forest

rf = RandomForestRegressor(n_estimators=50, 
                           criterion='mse', 
                           random_state=42, 
                           n_jobs=-1)
rf.fit(X_train_trans, y_train.ravel())

y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

In [ ]:
RMSE = 1043.202**0.5
RMSE

In [ ]:
df_final.listing_id.nunique()

In [ ]:
# Double check on price range. The range is as expected

y.describe(include='all')

In [ ]:
# Map feature importance with labels

feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [ ]:
# Order features based on importance

feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)
feature_lists

In [ ]:
features = [list_[0] for list_ in feature_lists]
features

In [ ]:
columns =['listing_id']
columns.extend(features)
columns.extend(['ref_district', 'ref_jurisdiction', 'price_by_month_week'])

In [ ]:
df_data = df_final[columns]
df_data.head()

In [ ]:
df_data.shape

In [ ]:
df_data.rename(columns={'price_by_month_week':'price'}, inplace=True)

In [ ]:
df_data.isnull().sum()

In [ ]:
temp_df = df_data[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].copy()

days = temp_df.apply(lambda x: x.idxmax(), axis=1)
days

In [ ]:
csv_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

csv_path = csv_file.format('airbnb_data')

df_data.to_csv(csv_path)

In [ ]:
# This is to create visual to show seasonality

df_visual = pd.merge(left=df_pricing_by_month_weekday, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [ ]:
df_visual.shape

In [ ]:
df_visual.columns

In [ ]:
df_visual_entire_apartment = df_visual[df_visual['room_type_Entire home/apt']==1 ]
df_visual_single_room = df_visual[df_visual['room_type_Entire home/apt']!=1 ]

print(df_visual_entire_apartment.shape, df_visual_single_room.shape )

In [ ]:
df_visual_entire_apartment.ref_district.describe

In [ ]:
df_visual_entire_apartment_sf = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='San Francisco']
df_visual_entire_apartment_scz = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='Santa Clara']

print(df_visual_entire_apartment_sf.shape, df_visual_entire_apartment_scz.shape )

In [ ]:
df_visual_entire_apartment_sf.id.nunique()

In [ ]:
df_visual_entire_apartment_scz.id.nunique()

In [ ]:
# Weekly pattern for entire apartment

df_week_entire_apartment = df_visual_entire_apartment.groupby(['ref_district', 'day_of_week'])  \
                                                     .price_by_month_week.mean()
df_week_entire_apartment

In [ ]:
# Weekly pattern for single room

df_week_single_room = df_visual_single_room.groupby(['ref_district', 'day_of_week'])  \
                                           .price_by_month_week.mean()
df_week_single_room

In [ ]:
# Montly Pattern for entire apartment

df_visual_entire_apartment.groupby(['ref_district', 'month']).price_by_month_week.mean()

In [ ]:
# Montly Pattern for single room

df_visual_single_room.groupby(['ref_district', 'month']).price_by_month_week.mean()